<a href="https://colab.research.google.com/github/ANIR1604/LinkedIn_JobFinder/blob/main/LinkedIN_JobFinder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [ ]:
title = input("Enter a job title: ")
location = input("Enter a location: ")
start = 0
end = int(input("Enter the number of pages you want to scrap: "))

In [ ]:
all_jobs = []
while start < end:
    list_url = f"https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords={title}&location={location}&start={start}"
    response = requests.get(list_url)
    list_data = response.text
    list_soup = BeautifulSoup(list_data, "html.parser")
    page_jobs = list_soup.find_all("li")
    for job in page_jobs:
        all_jobs.append(job)

    start += 1

In [ ]:
user_id = []

In [ ]:
for job in all_jobs:
    base_card_div = job.find("div", {"class": "base-card relative w-full hover:no-underline focus:no-underline base-card--link base-search-card base-search-card--link job-search-card"})
    job_id = base_card_div.get("data-entity-urn").split(":")[3]
    user_id.append(job_id)

In [ ]:
job = []
for job_id in user_id:
    job_url = f"https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/{job_id}"
    job_response = requests.get(job_url)
    job_soup = BeautifulSoup(job_response.text, "html.parser")
    job_post = {}
    try:
        job_post["Job Title"] = job_soup.find("h2", {"class":"top-card-layout__title font-sans text-lg papabear:text-xl font-bold leading-open text-color-text mb-0 topcard__title"}).text.strip()
    except:
        job_post["Job Title"] = None
    try:
        job_post["Company Name"] = job_soup.find("a", {"class": "topcard__org-name-link topcard__flavor--black-link"}).text.strip()
    except:
        job_post["Company Name"] = None
    try:
        job_post["Location"] = job_soup.find("span", {"class": "topcard__flavor topcard__flavor--bullet"}).text.strip()
    except:
        job_post["Location"] = None
    try:
        job_post["Time Posted"] = job_soup.find("span", {"class": "posted-time-ago__text topcard__flavor--metadata"}).text.strip()
    except:
        job_post["Time Posted"] = None
    try:
        job_post["Num Applicants"] = job_soup.find("figcaption", {"class": "num-applicants__caption"}).text.strip()
    except:
        job_post["Num Applicants"] = None
    job.append(job_post)

In [ ]:
jobs_df = pd.DataFrame(job)
jobs_df